<a href="https://colab.research.google.com/github/dmr1725/final-data-science/blob/master/proyecto_final_data_science.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a reproduction from https://github.com/c2-d2/pr_mort_official
## Adjusting for the missing households of size 1

If your code doesn't run, you need the following files under the sample_data directory:
1. death.RDS
2. hh_main.RDS
3. ind_hh.RDS
4. official_long.RDS
5. deaths_official.RDS
6. ACS2016.Rdata
7. adj_rates.RDS

These files can be found in the above github link under the data directory. 

We start by importing the necessary libraries.


In [ ]:
library(tidyverse)
library(lubridate)
options(digits = 3)

Now we import the data

In [ ]:
deaths <- readRDS("./sample_data/deaths.RDS")
households <- readRDS("./sample_data/hh_main.RDS")
individuals <- readRDS("./sample_data/ind_hh.RDS")
official <- readRDS("./sample_data/official_long.RDS")

Here, the after period of the hurricane is shorter than the before period. For our analysis, we will divide the fraction of year rather than the entire year. 

In [ ]:
days_before_hurricane <- 
  difftime(ymd("2017-09-20"), ymd("2016-12-31"), units = "days") %>% 
  as.numeric() 
years_before <- days_before_hurricane/365
years_after <- (365 - days_before_hurricane) / 365
years_before
years_after

[1] 0.721

[1] 0.279

Below we calculate the before and after death rates by households. Also, we calculate the median of the median ages of each household for each of these groups. 

In [ ]:
hh_stats <- households %>% 
  dplyr::select(hh_id, hh_size) %>% 
  filter(!is.na(hh_size) & hh_size>0) 

hh_deaths <- deaths %>%
  mutate(death_after = mo > 9.1 ) %>%
  group_by(hh_id) %>%
  summarize(tot_before = sum(!death_after), tot_after = sum(death_after)) %>%
  ungroup()

hh_deaths$hh_id <- as.numeric(hh_deaths$hh_id)
hh_stats$hh_id <- as.numeric(hh_stats$hh_id)

Here we calculate the per household size group rates. We will divide into five groups to make sure that the death rates before the hurricane are low compared to after the hurricane.

In [ ]:
rates_by_hh_size <- left_join(hh_stats, hh_deaths, by = "hh_id") %>%
  mutate(household_size = cut(hh_size, c(0,1,2,4,Inf), labels = c("1","2","3-4","5+"))) %>%
  group_by(household_size) %>%
  summarize(total_households = n(),
            N = sum(hh_size),
            deaths_before = sum(tot_before, na.rm = TRUE),
            deaths_after = sum(tot_after, na.rm = TRUE)) %>%
  mutate(rate_before = deaths_before/N*1000 / years_before,
         rate_after = deaths_after/N*1000 / years_after)
rates_by_hh_size %>% knitr::kable()



|household_size | total_households|    N| deaths_before| deaths_after| rate_before| rate_after|
|:--------------|----------------:|----:|-------------:|------------:|-----------:|----------:|
|1              |              534|  534|             0|            0|       0.000|       0.00|
|2              |             1074| 2148|             9|           15|       5.815|      24.99|
|3-4            |             1255| 4290|             8|           16|       2.588|      13.35|
|5+             |              436| 2550|             1|            7|       0.544|       9.82|

A problem that we're having is that the death rate of a household of 1 person is going to be 0. The reason for this is because if the only person that lives in a household dies, there is no other people to interview at that house. 

To avoid this problem, we will make a calculation that will treat the before and after death rate of a household of size 1 to have no effect on our analysis. Here, we compute the before and after September 20 rates for each year:

In [ ]:
population_by_year <- readRDS("./sample_data/deaths_official.RDS") %>%
  dplyr::select(Year, Popv17)
names(population_by_year) <- c("year","pop")
days_in_month <- c(31,28,31,30,31,30,31,31,30,31,30,31)

official_rates <- official %>% left_join(population_by_year, by = "year") %>%
  mutate(days = days_in_month[month]) %>%
  mutate(days = ifelse(year %% 4 == 0 & month ==2, days + 1, days)) %>%
  mutate(rate = deaths / pop * 365/days * 1000) %>%
  group_by(year) %>%
  summarize(before_rate = sum(deaths*(month<9) + deaths*(month==9)*2/3) / pop[1] * 1000 *
              sum(days)/(sum(days*(month<9))+20),
            after_rate = sum(deaths*(month>9) + deaths*(month==9)*1/3) / pop[1] * 1000 *
              sum(days)/(sum(days*(month>9))+10))

Let's quickly check the rate estimate (full estimate and CI in excess_est.R)


In [ ]:
pr_pop <- population_by_year %>% subset(year == 2016) %>% {.$pop}
  
res <- rates_by_hh_size %>% 
  summarize(survey_deaths = sum(deaths_after), 
            N = sum(N), 
            rate = round(survey_deaths/N*1000/years_after,1))

t(res) 

survey_deaths,38.0
N,9522.0
rate,14.3


Here we take in consideration the 2017 before rate for both before and after rates for the 1 person households:

In [ ]:
rates_by_hh_size_adj <- rates_by_hh_size
rates_by_hh_size_adj$rate_before[1] <- rates_by_hh_size_adj$rate_after[1] <- 
  official_rates %>%
  filter(year == 2017) %>% 
  .$before_rate
rates_by_hh_size_adj %>% knitr::kable()



|household_size | total_households|    N| deaths_before| deaths_after| rate_before| rate_after|
|:--------------|----------------:|----:|-------------:|------------:|-----------:|----------:|
|1              |              534|  534|             0|            0|       8.847|       8.85|
|2              |             1074| 2148|             9|           15|       5.815|      24.99|
|3-4            |             1255| 4290|             8|           16|       2.588|      13.35|
|5+             |              436| 2550|             1|            7|       0.544|       9.82|

Based on our problem above (households of 1 size), the survey of Dr. Irizarry is higher for larger households. 

The ACS2016 data can confirm this. 

In [ ]:
load("./sample_data/ACS2016.Rdata")
household_dist <- acs.hh_size %>% 
  mutate(hh_size = cut(hh_size, c(0,1,2,4,Inf), labels = c("1","2","3-4","5+"))) %>%
  group_by(hh_size)%>%
  summarize(count = sum(count)) %>%
  ungroup() %>%
  mutate(pop_freq = count / sum(count)) 

We can also adjust for this and obtain the following before and after rates:


In [ ]:
res_2 <- rates_by_hh_size_adj %>%  left_join(household_dist, by = c("household_size" = "hh_size")) 
res_2 %>%
  summarize(rate_before = sum(rate_before*pop_freq), 
            se_before = sum(rate_before*pop_freq)/sqrt(sum(deaths_before)),
            rate_after = sum(rate_after*pop_freq), 
            se_after = sum(rate_after*pop_freq)/sqrt(sum(deaths_after))) %>%
  mutate(lower_before = rate_before - 1.96*se_before, 
         upper_before = rate_before+ 1.96*se_before,
         lower_after = rate_after - 1.96*se_after, 
         upper_after = rate_after+ 1.96*se_after) -> adj_rates

saveRDS(adj_rates, "./sample_data/adj_rates.RDS")

t(adj_rates)

rate_before,5.08
se_before,1.20
rate_after,15.55
se_after,2.52
lower_before,2.73
upper_before,7.42
lower_after,10.61
upper_after,20.50
